In [1]:
import pandas as pd

In [148]:
data = pd.read_csv('./data/gc_20_9',sep=' ')
node_count = int(data.columns[0])
edge_count = int(data.columns[1])
data = data.rename(columns={data.columns[0]:'from',data.columns[1]:'to'})

In [149]:
voisins = data.groupby('from').apply(lambda x: set(x['to'].values))
voisins = voisins.loc[range(node_count)].apply(lambda x:  set() if type(x) is not set else x)

In [150]:
idx = voisins.apply(lambda x: len(x)).sort_values(ascending=False).index

In [151]:
def graph_coloring_v1(nodes, voisins, rank_nodes, visited_nodes = set(), rank = 0):
    if nodes == set():
        return visited_nodes, rank_nodes
    for n in nodes:
        if n not in visited_nodes:
            #print 'node %d rank %d'%(n,rank)
            rank_nodes[n]=rank
            visited_nodes |= set([n])
        adj_nodes = voisins[n]-visited_nodes
        for n2 in adj_nodes:
            #print 'node %d rank %d'%(n2,rank+1)
            rank_nodes[n2]=rank+1
        visited_nodes |= set(adj_nodes)
        visited_nodes, rank_nodes = graph_coloring_v1(adj_nodes, 
                                                      voisins, 
                                                      rank_nodes, 
                                                      visited_nodes, 
                                                      rank + 1)
    
    #print rank_nodes

    return visited_nodes, rank_nodes

In [152]:
nodes = voisins.index.tolist()
rank_nodes = [-1]*node_count
visited_nodes, rank_nodes = graph_coloring_v1(nodes,voisins,rank_nodes)

In [153]:
rank_nodes

[0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [162]:
voisins[4]

{5, 6, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}